In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 4567.03it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:06<00:00, 2952.25it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 1977559.23it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 5080.39it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:06<00:00, 2903.71it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 44426.00it/s]


In [5]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 2851.60it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 2139.99it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 44791.63it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 5058.29it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 2941.21it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 45273.81it/s]


In [7]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [8]:
bin_size = 30
model = mz.models.DRMM()
model.params['input_shapes'] = [[10,], [10, bin_size,]]
model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5002200     text_left[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
__________________________________________________________________________________________________
attention_

In [10]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [11]:
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [12]:
model.load_embedding_matrix(embedding_matrix)

In [13]:
pred_generator = mz.HistogramDataGenerator(data_pack=predict_pack_processed,
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, 
                                           x=pred_x, 
                                           y=pred_y, 
                                           once_every=1, 
                                           batch_size=len(pred_y),
                                           model_save_path='./drmm_pretrained_model/'
                                          )

In [14]:
train_generator = mz.HistogramPairDataGenerator(train_pack_processed, embedding_matrix, bin_size, 'LCH',
                                                num_dup=2, num_neg=4, batch_size=20)
len(train_generator)

102

In [15]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 9s 86ms/step - loss: 1.5377
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4756536327915165 - normalized_discounted_cumulative_gain@5(0.0): 0.552560960894608 - mean_average_precision(0.0): 0.5020347452745515
Epoch 2/30
102/102 [==============================] - 7s 64ms/step - loss: 1.4151
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5753281546804131 - normalized_discounted_cumulative_gain@5(0.0): 0.6353491840238532 - mean_average_precision(0.0): 0.5966143093767571
Epoch 3/30
102/102 [==============================] - 8s 74ms/step - loss: 1.2734
Epoch 3/30
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5753281546804131 - normalized_discounted_cumulative_gain@5(0.0): 0.6353491840238532 - mean_average_precision(0.0): 0.5966143093767571
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5969699060288235 - normalized_discounted_cumulative_gain@5(0.0): 0.658063367513948 - mean_average_prec

102/102 [==============================] - 8s 75ms/step - loss: 0.6094
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.627531716945488 - normalized_discounted_cumulative_gain@5(0.0): 0.6769197039119518 - mean_average_precision(0.0): 0.6341693826396657
Epoch 23/30
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6200477270110192 - normalized_discounted_cumulative_gain@5(0.0): 0.6744459622116895 - mean_average_precision(0.0): 0.6309813845149587
Epoch 24/30
102/102 [==============================] - 7s 72ms/step - loss: 0.6041
Epoch 24/30
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6260187766210515 - normalized_discounted_cumulative_gain@5(0.0): 0.6776703124693056 - mean_average_precision(0.0): 0.6369712824166213
Epoch 25/30
101/102 [============================>.] - ETA: 0s - loss: 0.5976
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6260187766210515 - normalized_discounted_cumulative_gain@5(0.0): 0.6776703124693056 - mean_average_prec

In [16]:
drmm_model = mz.load_model('./drmm_pretrained_model/16')
test_generator = mz.HistogramDataGenerator(data_pack=valid_pack_processed[:10],
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
test_x, test_y = test_generator[:]
prediction = drmm_model.predict(test_x)
prediction

array([[ 1.5303962 ],
       [ 0.30404466],
       [ 1.592996  ],
       [ 1.6210198 ],
       [ 0.26755413],
       [-0.8538296 ],
       [-2.118522  ],
       [ 0.6409646 ],
       [-2.8159575 ],
       [ 1.0737036 ]], dtype=float32)

In [17]:
import shutil
shutil.rmtree('./drmm_pretrained_model/')